## **Importing Packages**

In [ ]:
!pip install shap
!pip install eli5
from google.colab import drive

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
shap.initjs()
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectFromModel
from statsmodels.stats.multitest import multipletests
from sklearn.feature_selection import f_classif
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.pipeline import make_pipeline
import eli5
from eli5.sklearn import PermutationImportance
from itertools import combinations
from functools import reduce
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import mutual_info_classif
import itertools
from scipy import stats
from sklearn.svm import SVC
import networkx as nx
import os
import zipfile
from google.colab import files
from scipy.stats import mannwhitneyu
from collections import defaultdict
protein_coding_genes = pd.read_csv('mart_export.txt', sep='\t')
protein_coding_genes = list(set(list(protein_coding_genes["Gene name"])))

## **Helper Functions**

In [ ]:
'''
A function that pre-processes gene expression data.
It takes as an input a gene expression dataframe, some other parameter, and returns a filtered, pre-processed gene expression dataframe.
'''

def GeneExpressionPreprocessing(mRNAFile, DropList, keepList, GeneColumnName, Need_Transpose=True, Percent_To_FilterOut=20, tpm=False, filter_genes_with_low_expression=False):
  filtered_data = mRNAFile
  for i in DropList: # Dropping columns
    filtered_data = mRNAFile.drop(i, axis=1, errors='ignore')
  if Need_Transpose == True: # transpose the file such that genes are columns and samples are rows
    filtered_data = filtered_data.set_index(GeneColumnName)
    filtered_data = filtered_data.T
    filtered_data = filtered_data.rename_axis('Sample', axis=1)
  if tpm:
    for row in filtered_data.index:
      row_sum = filtered_data.loc[row].sum()
      for column in filtered_data.columns:
        filtered_data.loc[row, column] = (filtered_data.loc[row, column] / row_sum) * 10**6

  if filter_genes_with_low_expression:
    # Filter out genes which are zeroes in certain percentage from the data.
    filtered_data = filtered_data.loc[:, (filtered_data == 0).mean() * 100 <= Percent_To_FilterOut]

  filtered_data = filtered_data.reset_index()
  filtered_data = filtered_data.rename(columns={'index': 'Sample Identifier'})

  filtered_columns = ['Sample Identifier'] + [col for col in set(filtered_data.columns[1:]) if col in keepList]
  filtered_data = filtered_data[filtered_columns]

  return filtered_data

In [ ]:
'''
A function that merges columns based on a common column.
It takes as an input a list of all the dataframes that should be merged and the merging column
This function is very important as we will use it often later
'''

def merge_on_common_column(dataframes, merge_column):
    merged_df = dataframes[0]
    for df in dataframes[1:]:
        merged_df = pd.merge(merged_df, df, on=merge_column)
    return merged_df

'''
A function that pre-processes clinical data.
It takes as an input a clinical dataframe, some other parameters, and returns a filtered, pre-processed clinical dataframe.
'''
def ClinicalPreProcessing(clinicalData, ColumnsToKeep, columnsToEncode, columnsToFillMean, categoricalVal):
  clinical = clinicalData[ColumnsToKeep] # keeps only desired columns
  clinical = clinical.dropna(subset=[col for col in clinical.columns if col not in columnsToFillMean]) # drops columns that have nulls if they are not to be filled with the mean
  for i in columnsToEncode:
    clinical[i] = (clinical[i] == categoricalVal).astype(int)
    clinical[i] = clinical[i].astype('category') # encoding categorical data --> making sure that 1's and 0's, or similar data, are treated as categorical
  for i in columnsToFillMean:
    clinical[i] = pd.to_numeric(clinical[i], errors='coerce')
    mean_col = clinical[i].mean()
    clinical[i].fillna(mean_col, inplace=True)
  return clinical

'''
A function that normalizes a dataframe with the MinMaxScaler.
'''
def normalize(df, include_last_col = False):
  norm_df = df
  final_num = len(df.columns.tolist())
  if not(include_last_col):
    final_num = -1
  for i in norm_df.columns.tolist()[1:final_num]:
    if norm_df[i].dtype == 'object':
      norm_df[i] = pd.to_numeric(norm_df[i], errors='coerce')
  numeric_cols = norm_df.select_dtypes(include=[np.number]).columns.tolist()
  for col in numeric_cols:
      if not norm_df[col].isnull().all():
          scaler = MinMaxScaler()
          norm_df[col] = scaler.fit_transform(norm_df[[col]])
  return norm_df

In [ ]:
'''
A function that converts SNP mutation data (MAF files) to a SNP count matrix.
It takes as an input a mutation dataframe, a column for samples, and a column for genes. It returns a SNP mutation counts dataframe.
'''

def create_count_matrix(data, sample_col, gene_col):
    unique_samples = data[sample_col].unique() # getting all unique samples
    unique_genes = data[gene_col].unique() # getting all unique genes
    count_matrix  = pd.DataFrame(0, index=unique_samples, columns=unique_genes) # creating an initial dataframe
    for _, row in data.iterrows():
        sample = row[sample_col]
        gene = row[gene_col]
        count_matrix.at[sample, gene] += 1 # adding a 1 whenever a gene-sample combination has occured

    count_matrix.reset_index(inplace=True)
    count_matrix = count_matrix.rename(columns={'index': 'Sample'}) # renaming index to Sample
    count_matrix.columns = [col + '_SNP' for col in count_matrix.columns] # this is needed for building an integrative model later where the same gene has expression, SNP mutation, and CNA mutation as features

    columns_to_convert = count_matrix.columns[1:]
    for column in columns_to_convert:
      count_matrix[column] = count_matrix[column].astype('int')
    count_matrix = count_matrix.rename(columns={'Sample_SNP': 'Sample Identifier'})
    return count_matrix

In [ ]:
'''
A function that pre-processes CNA mutation data.
It takes as an input a CNA mutation dataframe, some other parameters, and returns a processed CNA mutation dataframe.
'''

def process_and_rename_columns(data, unique_col, cat=True):
    data_processed = data.drop_duplicates(subset=unique_col, keep='first') # Dropping duplicates
    data_processed.reset_index(drop=True, inplace=True)
    data_processed = data_processed.T # Transposing to be compatible with later analyses
    data_processed.columns = data_processed.iloc[0]
    data_processed = data_processed.drop(data_processed.index[0])
    data_processed.columns = [col + '_CN' for col in data_processed.columns] # this is needed for building an integrative model later where the same gene has expression, SNP mutation, and CNA mutation as features
    if cat: # converting to categorical if the numbers in the CNA dataframer indicate groups.
      columns_to_convert = data_processed.columns
      for column in columns_to_convert:
        data_processed[column] = data_processed[column].astype('category')
    data_processed = data_processed.reset_index()
    data_processed = data_processed.rename(columns={'index': 'Sample Identifier'})
    return data_processed

In [ ]:
'''
A function that pre-processes cibersort data
'''
def processCibersort(df):
  df = df.iloc[:, :-4]

  numeric_cols = df.select_dtypes(include=[np.number]).columns

  df[numeric_cols] = MinMaxScaler().fit_transform(df[numeric_cols])

  df.rename(columns={'Mixture': 'Sample Identifier'}, inplace=True)

  return df

In [ ]:
'''
A function that merges more than two dataframes.
'''

def merge_dataframes(dataframeslist, commoncolumn):
  return reduce(lambda left, right: pd.merge(left, right, on=commoncolumn, how='inner'), dataframeslist)

In [ ]:
'''
A function to convert continuous CNA data to categorical according to certain thresholds.
'''
def CNA_threshold(value):
  if value < 0.1 and value >= 0 :
    return 0.0
  elif value > 0.9 :
    return 2.0
  elif value > 0.1 and value < 0.9 :
    return 1.0
  elif value > -0.1 and value <= 0 :
    return 0.0
  elif value < -1.3 :
    return -2.0
  elif value < -0.1 and value > -1.3 :
    return -1.0

In [ ]:
def remove_duplicates(df, duplicateList, columnToMergeOn= 'Sample Identifier'):
  duplicateset = set(duplicateList)
  df_final = df.drop(list(duplicateset), axis=1)
  for duplicate in duplicateset:
    DuplicatesToKeep = [True]
    gene = df[[columnToMergeOn, duplicate]]
    col_median = gene[duplicate].median()
    for median in col_median:
      if median == max(col_median):
        DuplicatesToKeep.append(True)
      else:
        DuplicatesToKeep.append(False)
    gene = gene.iloc[:,DuplicatesToKeep]
    df_final = merge_on_common_column([df_final, gene], columnToMergeOn)
  return df_final

In [ ]:
def plot_confusion(y_test, y_pred, confusion_matrix_path, archive):
  cm = confusion_matrix(y_test, y_pred)
  plt.figure(figsize=(10, 8))
  sns.heatmap(cm, annot=True, fmt="d", cmap='Blues')
  plt.xlabel('Predicted')
  plt.ylabel('True')
  plt.title(f'Confusion Matrix for {confusion_matrix_path}')
  plt.savefig(confusion_matrix_path)
  plt.close()

## **All neccessary imports of files**

In [ ]:
'''
LIU DATA
'''

# Raw files, before pre-processing
clinical_patientLiu = pd.read_csv('/Liu/data_clinical_patient.txt', sep='\t').iloc[4:]
clinical_samplecLiu = pd.read_csv('/Liu/data_clinical_sample.txt', sep='\t').iloc[4:]
data_cnaLiu = pd.read_csv('/Liu/data_cna.txt', sep='\t')
data_mrnaLiu = pd.read_csv('/Liu/data_mrna_seq_tpm.txt', sep='\t')
data_mutationscLiu = pd.read_csv('/Liu/data_mutations.txt', sep='\t')

# pre-processed files
LabelsLiu = pd.read_csv('/Liu/labels_cbioportal.tsv', sep='\t')
CNADataLiu = pd.read_csv('/Liu/processed_cna_data_liu.tsv', sep='\t')
GeneExpressionDataLiu = pd.read_csv('/Liu/processed_exp_data_liu.tsv', sep='\t')
SNPDataLiu = pd.read_csv('/Liu/processed_snp_data_liu.tsv', sep='\t')
ClinicalDataLiu = pd.read_csv('/Liu/processed_clinical_data_liu.tsv', sep='\t')
ProcessedImmuneLiu = processCibersort(pd.read_csv('/Liu/CIBERSORTx.csv', sep=','))

'''
RAVI DATA
'''
clinical1Ravi = pd.read_excel('/Ravi/clinical.xlsx')
clinical2Ravi = pd.read_csv('/Ravi/antigen_presentation.tsv', sep='\t')
clinical3Ravi = pd.read_csv('/Ravi/purity_ploidy.tsv', sep='\t')
clinical4Ravi = pd.read_csv('/Ravi/total_amps_dels.tsv', sep='\t')
clinical5Ravi = pd.read_csv('/Ravi/tmb.tsv', sep='\t')
data_cnaRavi = pd.read_csv('/Ravi/gistic_gene.tsv', sep='\t')
data_mrnaRavi = pd.read_csv('/Ravi/mrna_tpm.tsv', sep='\t')
data_mutationsRavi = pd.read_csv('/Ravi/Absolute_MAF_Final.tsv', sep='\t')

# pre-processed files
LabelsRavi = pd.read_csv('/Ravi/labels_ravi.tsv', sep='\t')
CNADataRavi = pd.read_csv('/Ravi/processed_cna_data_ravi.tsv', sep='\t')
GeneExpressionDataRavi = pd.read_csv('/Ravi/processed_exp_data_ravi.tsv', sep='\t')
SNPDataRavi = pd.read_csv('/Ravi/processed_snp_data_ravi.tsv', sep='\t')
ClinicalDataRavi = pd.read_csv('/Ravi/processed_clinical_data_ravi.tsv', sep='\t')

'''
MERGED DATA
'''
merged_clinical = pd.read_csv('/Merged/merged_clinical.tsv', sep='\t')
merged_labels = pd.read_csv('/Merged/merged_labels.tsv', sep='\t')
merged_expression = pd.read_csv('/Merged/merged_expression.tsv', sep='\t')
merged_snp = pd.read_csv('/Merged/merged_snp.tsv', sep='\t')
merged_snp_pre_normalization = pd.read_csv('/Merged/merged_snp_pre_normalization.tsv', sep='\t')
merged_cna = pd.read_csv('/Merged/merged_cna.tsv', sep='\t')
merged_cibersort = processCibersort(pd.read_csv('/Merged/cibersort_merged.csv', sep=','))


rf_integrative = pd.read_csv('/Merged/FS/rf_integrative.tsv', sep='\t')
rfe_integrative = pd.read_csv('/Merged/FS/rfe_integrative.tsv', sep='\t')
fdr_integrative = pd.read_csv('/Merged/FS/fdr_integrative.tsv', sep='\t')
MI_integrative = pd.read_csv('/Merged/FS/MI_integrative.tsv', sep='\t')

for column in merged_cna.columns.tolist()[1:]:
        merged_cna[column] = merged_cna[column].astype('category')

'''
VALIDATION DATA
'''
# Raw files, before pre-processing
clinical_patient_validation = pd.read_csv('/Validation/data_clinical_patient_validation.txt', sep='\t').iloc[4:]
clinical_sample_validation = pd.read_csv('/Validation/data_clinical_sample_validation.txt', sep='\t').iloc[4:]
data_cna_validation = pd.read_csv('/Validation/data_cna.txt', sep='\t')
data_mrna_validation = pd.read_csv('/Validation/data_mrna_seq_rpkm.txt', sep='\t')
data_mutations_validation = pd.read_csv('/Validation/data_mutations.txt', sep='\t')
mapping_validation = pd.read_csv('/Validation/mapping_from_entrez_to_symbol_for_validation_data.tsv', sep='\t')

# pre-processed files
LabelsValidation = pd.read_csv('/Validation/labels_validation.tsv', sep='\t')
CNADataValidation = pd.read_csv('/Validation/processed_cna_data_validation.tsv', sep='\t')
GeneExpressionDataValidation = pd.read_csv('/Validation/processed_exp_data_validation_unfiltered_with_zero_expression.tsv', sep='\t')
SNPDataValidation = pd.read_csv('/Validation/processed_snp_data_validation.tsv', sep='\t')
ClinicalDataValidation = pd.read_csv('/Validation/processed_clinical_data_validation.tsv', sep='\t')
cibersort_validation = processCibersort(pd.read_csv('/Validation/cibersort_validation.csv', sep=','))

## **Pre-processing Liu Data**

In [ ]:
merged_clinicalLiu = merge_on_common_column([clinical_samplecLiu, clinical_patientLiu], '#Patient Identifier')

# SPECIFIC TO THE FIRST DATA
# .............................................

# Solving issues in column names
immunotherapy_mapping = {
    'MK3475': 'Pembrolizumab',
    'MK 3475': 'Pembrolizumab',
    'Nivo': 'Nivolumab',
    'Nivolumab': 'Nivolumab',
    'nivolumab': 'Nivolumab',
    'Pembro': 'Pembrolizumab',
    'Pembrolizumab': 'Pembrolizumab',
    'pembrolizumab': 'Pembrolizumab',
}
merged_clinicalLiu['Immunotherapy'] = merged_clinicalLiu['Immunotherapy'].map(immunotherapy_mapping)
# Perform one-hot encoding for 'Immunotherapy' column
merged_clinicalLiu = pd.get_dummies(merged_clinicalLiu, columns=['Immunotherapy'])
# ..........................................................................

labelsLiu = merged_clinicalLiu[['Sample Identifier', 'Best Radiographic Response (RECIST 1.1)']]
response_mapping = {
    'Complete Response': 1,
    'Partial Response': 1,
    'Stable Disease': 0,
    'Progressive Disease': 0
}
labelsLiu['Best Radiographic Response (RECIST 1.1)'] = labelsLiu['Best Radiographic Response (RECIST 1.1)'].map(response_mapping)
labelsLiu = labelsLiu.dropna(subset = ['Best Radiographic Response (RECIST 1.1)', 'Sample Identifier'])
labelsLiu = labelsLiu.rename(columns={'Best Radiographic Response (RECIST 1.1)': 'ICB Response'})
labelsLiu['ICB Response'] = labelsLiu['ICB Response'].astype('category')
filter_idsLiu = labelsLiu['Sample Identifier']

# Preparing all the parameters needed for pre-processing cBioportal clinical data

columnsToFillNANWithMeanLiu = ['LDH (treatment Start)']
ImportantClinicalFeaturesLiu = ['Sample Identifier', '#Patient Identifier', 'LDH (treatment Start)', 'Total Mutations',
       'Nonsynonymous Mutation Count', 'Mutations Clonal', 'Mutations Subclonal', 'Heterogeneity', 'Total Neoantigen', 'CNA Prop', 'Purity', 'Ploidy',
       'TMB (nonsynonymous)', 'Brain Metastasis', 'Cutaneous Subcutaneous Metastasis', 'LN Metastasis', 'Lung Metastasis', 'Liver Visceral Metastasis', 'Bone Metastasis',
       'Alkylating Chemotherapy',
       'Immunotherapy_Nivolumab', 'Immunotherapy_Pembrolizumab']
FeaturesNeedEncodingLiu = ['Brain Metastasis', 'Cutaneous Subcutaneous Metastasis', 'LN Metastasis', 'Lung Metastasis', 'Liver Visceral Metastasis', 'Bone Metastasis']
ProcessedClinicalDataLiu = ClinicalPreProcessing(merged_clinicalLiu, ImportantClinicalFeaturesLiu, FeaturesNeedEncodingLiu, columnsToFillNANWithMeanLiu, "Yes")
for i in ['Immunotherapy_Nivolumab',	'Immunotherapy_Pembrolizumab']:
    ProcessedClinicalDataLiu[i] = (ProcessedClinicalDataLiu[i] == True).astype(int)
    ProcessedClinicalDataLiu[i] = ProcessedClinicalDataLiu[i].astype('category')
ProcessedClinicalDataLiu = ProcessedClinicalDataLiu.drop('#Patient Identifier', axis=1)
ProcessedClinicalDataLiu = ProcessedClinicalDataLiu[ProcessedClinicalDataLiu['Sample Identifier'].isin(filter_idsLiu)]

In [ ]:
ProcessedClinicalDataLiu

,Sample Identifier,LDH (treatment Start),Total Mutations,Nonsynonymous Mutation Count,Mutations Clonal,Mutations Subclonal,Heterogeneity,Total Neoantigen,CNA Prop,Purity,...,TMB (nonsynonymous),Brain Metastasis,Cutaneous Subcutaneous Metastasis,LN Metastasis,Lung Metastasis,Liver Visceral Metastasis,Bone Metastasis,Alkylating Chemotherapy,Immunotherapy_Nivolumab,Immunotherapy_Pembrolizumab
0,Sample1,323.269504,34,22,12,10,0.454545455,49,0.321417021,0.92,...,0.666666666667,0,1,0,1,1,1,2.51455596,0,1
1,Sample10,2186.000000,96,71,48,22,0.314285714,230,0.391383951,0.83,...,2.36666666667,0,1,1,1,1,1,6.81073463,0,1
2,Sample100,313.000000,200,126,98,24,0.196721311,301,0.02944673,0.11,...,4.2,0,1,1,0,0,0,12.6984645,1,0
4,Sample104,270.000000,130,96,65,28,0.301075269,329,0.206518421,0.86,...,3.2,0,0,1,0,0,0,3.58768501,0,1
6,Sample106,403.000000,380,245,194,36,0.156521739,726,0.22414699,0.62,...,8.23333333333,0,1,0,0,0,0,27.3833807,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Sample9,192.000000,911,652,562,80,0.124610592,1996,0.111034165,0.88,...,21.8,0,1,1,0,0,0,36.3703376,1,0
140,Sample94,817.000000,414,268,227,21,0.084677419,929,0.902095292,0.85,...,8.9,1,1,1,1,1,1,2.28E-08,0,1
141,Sample96,174.000000,959,649,588,50,0.078369906,2136,0.196499246,0.82,...,21.4666666667,0,0,0,1,1,0,2.37349696,1,0
142,Sample98,192.000000,472,302,219,74,0.252559727,790,0.223074401,0.6,...,10.0666666667,0,0,1,1,0,0,6.51309681,1,0


In [ ]:
ProcessedGeneExpressionDataLiu = GeneExpressionPreprocessing(data_mrnaLiu, ['Entrez_Gene_Id'], protein_coding_genes, 'Hugo_Symbol', filter_genes_with_low_expression=False)
ProcessedGeneExpressionDataLiu = ProcessedGeneExpressionDataLiu[ProcessedGeneExpressionDataLiu['Sample Identifier'].isin(filter_idsLiu)]

#len(ProcessedGeneExpressionDataLiu.columns) == len(set(ProcessedGeneExpressionDataLiu.columns)) --> False
# so, I continue with removing duplicates using the helper function above

columnsLiu = ProcessedGeneExpressionDataLiu.columns.tolist()
duplicated_columnsLiu = []
for column in columnsLiu:
  if columnsLiu.count(column) > 1:
    duplicated_columnsLiu.append(column)
ProcessedGeneExpressionDataLiu = remove_duplicates(ProcessedGeneExpressionDataLiu, duplicated_columnsLiu)


# len(ProcessedGeneExpressionDataLiu.columns) == len(set(ProcessedGeneExpressionDataLiu.columns)) --> True
# # _____________________________________________________

ProcessedSNPDataLiu = data_mutationsLiu[data_mutationsLiu['Hugo_Symbol'].isin(protein_coding_genes)]
ProcessedSNPDataLiu = create_count_matrix(ProcessedSNPDataLiu, 'Tumor_Sample_Barcode', 'Hugo_Symbol')
ProcessedSNPDataLiu = ProcessedSNPDataLiu[ProcessedSNPDataLiu['Sample Identifier'].isin(filter_idsLiu)]

# len(ProcessedSNPDataLiu.columns) == len(set(ProcessedSNPDataLiu.columns)) --> True
# _____________________________________________________

ProcessedCNADataLiu = data_cnaLiu[data_cnaLiu['Hugo_Symbol'].isin(protein_coding_genes)]
ProcessedCNADataLiu = process_and_rename_columns(ProcessedCNADataLiu, 'Hugo_Symbol')
ProcessedCNADataLiu = ProcessedCNADataLiu[ProcessedCNADataLiu['Sample Identifier'].isin(filter_idsLiu)]

# len(ProcessedCNADataLiu.columns) == len(set(ProcessedCNADataLiu.columns)) --> True

In [ ]:
ProcessedGeneExpressionDataLiu

Sample,Sample Identifier,ZNF423,FRS2,C3orf62,DDRGK1,DDX6,GPT,GOLGA4,ZFAND1,SERPINB12,...,SIVA1,MAMLD1,SLFN12L,CDKL4,TTC33,TP53BP2,TH,S1PR3,MYO18A,PALM2AKAP2
0,Sample100,6.702265,13.650808,33.282641,28.445048,71.666744,4.450586,79.653171,11.522267,0.000000,...,42.060674,10.748253,18.752270,0.211095,7.810514,33.476145,0.087956,3.359928,90.594927,62.290609
1,Sample106,2.114254,12.970594,7.696836,26.511324,77.776052,2.898191,71.314510,52.333733,0.000000,...,19.028289,8.908375,1.164028,1.164028,5.820138,25.964943,0.000000,0.878960,82.788497,23.541865
2,Sample107,0.000000,16.590427,0.000000,26.612863,68.498009,5.590747,201.244150,50.430352,0.000000,...,44.862332,0.000000,0.000000,0.000000,0.000000,166.790606,0.000000,0.000000,3.977157,32.362696
3,Sample108,0.000000,46.892543,3.712011,25.605298,32.196011,0.000000,31.286947,48.180384,0.000000,...,2.121149,20.908468,0.000000,0.000000,2.954457,17.044947,0.000000,0.000000,1.742372,2.424170
4,Sample10,5.631753,19.815428,7.205610,22.583899,75.033156,4.702609,70.235737,36.596915,0.000000,...,48.770603,9.310407,1.725554,0.132735,7.490042,30.832427,0.075849,1.687630,95.607069,49.965218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,Sample196,0.347341,16.305727,8.818600,61.305675,69.294517,1.254287,137.855759,27.111890,0.154374,...,32.900905,28.520550,0.096484,1.891078,15.379485,63.737062,0.347341,0.656088,79.811229,13.527000
115,Sample197,0.260142,10.125542,15.968740,57.751609,52.548761,13.367316,86.267216,20.931456,0.000000,...,40.081938,22.412267,0.220120,2.341281,11.886506,50.767786,0.000000,0.620340,58.652101,28.475585
116,Sample204,0.915825,19.482091,17.012139,28.335064,60.555446,11.933475,91.748993,25.837360,3.913070,...,27.696762,31.859602,1.248852,1.054586,10.684623,26.198140,0.055505,0.943577,47.650642,36.022442
117,Sample205,0.823763,18.045554,8.520797,25.176251,54.368346,3.449507,105.158472,9.988124,0.128713,...,50.146562,84.023807,1.029704,1.390100,9.061391,46.413886,0.360396,3.011883,55.449535,55.140624


In [ ]:
ProcessedSNPDataLiu

,Sample Identifier,LRP1B_SNP,EIF3H_SNP,CLEC18B_SNP,BCLAF1_SNP,MUC16_SNP,RAB11FIP1_SNP,ZNF276_SNP,PMFBP1_SNP,PPFIBP2_SNP,...,MT-ND1_SNP,MT-ND4_SNP,ZNF888_SNP,MT-ND2_SNP,C11orf52_SNP,NT5C3A_SNP,KLF6_SNP,MT-ND5_SNP,TMCO1_SNP,PROK2_SNP
1,Sample203,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Sample159,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Sample108,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Sample104,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,Sample21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Sample180,0,0,0,0,5,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
140,Sample121,0,0,0,0,5,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
141,Sample72,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
142,Sample59,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
ProcessedCNADataLiu

,Sample Identifier,A1BG_CN,A1CF_CN,A2M_CN,A2ML1_CN,A3GALT2_CN,A4GALT_CN,A4GNT_CN,AAAS_CN,AACS_CN,...,ZSWIM8_CN,ZW10_CN,ZWILCH_CN,ZWINT_CN,ZXDC_CN,ZYG11A_CN,ZYG11B_CN,ZYX_CN,ZZEF1_CN,ZZZ3_CN
0,Sample1,0,-1,0,0,-1,0,-1,0,0,...,-1,0,-1,-1,-1,-1,-1,0,0,-1
1,Sample10,0,-1,0,0,-1,0,0,0,-1,...,-1,0,0,-1,0,-1,-1,0,-1,-1
2,Sample100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Sample104,0,-1,0,0,0,0,0,0,0,...,-1,0,0,-1,0,0,0,0,0,0
6,Sample106,0,-1,0,0,-1,0,-1,0,0,...,-1,0,0,-1,-1,-1,-1,0,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Sample9,0,0,0,0,0,0,0,-1,-1,...,0,-1,0,0,0,0,0,0,0,0
140,Sample94,-1,-1,-1,-1,-1,0,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
141,Sample96,0,-1,0,0,0,0,0,0,0,...,-1,-1,0,-1,0,0,0,0,-1,0
142,Sample98,0,-1,0,0,-1,0,-1,0,0,...,-1,-1,0,-1,-1,-1,-1,0,0,-1


## **Pre-processing Ravi Data**

In [ ]:
clinical1Ravi['Harmonized_SU2C_WES_Tumor_Sample_ID_v2'] = clinical1Ravi['Harmonized_SU2C_WES_Tumor_Sample_ID_v2'].str.replace('-T', '')
merged_clinicalRavi = merge_on_common_column([clinical1Ravi, clinical2Ravi, clinical3Ravi, clinical4Ravi, clinical5Ravi], 'Harmonized_SU2C_WES_Tumor_Sample_ID_v2')
merged_clinicalRavi = pd.get_dummies(merged_clinicalRavi, columns=['Agent_PD1'])

labelsRavi = clinical1Ravi[['Harmonized_SU2C_WES_Tumor_Sample_ID_v2', 'Harmonized_Confirmed_BOR']]
labelsRavi = labelsRavi.rename(columns={'Harmonized_SU2C_WES_Tumor_Sample_ID_v2': 'Sample Identifier'})

response_mapping = {
    'CR': 1,
    'PR': 1,
    'SD': 0,
    'PD': 0
}
stage_mapping = {
    1.0: 0,
    2.0: 0,
    3.0: 1,
    4.0: 1
}
merged_clinicalRavi['Initial_Stage'] = merged_clinicalRavi['Initial_Stage'].map(stage_mapping)

labelsRavi['Harmonized_Confirmed_BOR'] = labelsRavi['Harmonized_Confirmed_BOR'].map(response_mapping)
labelsRavi = labelsRavi.dropna(subset = ['Harmonized_Confirmed_BOR', 'Sample Identifier'])
labelsRavi = labelsRavi.rename(columns={'Harmonized_Confirmed_BOR': 'ICB Response'})
labelsRavi['ICB Response'] = labelsRavi['ICB Response'].astype('category')
filter_idsRavi = labelsRavi['Sample Identifier']

columnsToFillNANWithMeanRavi = ['Patient_Smoking_Pack_Years_Harmonized']
ImportantClinicalFeaturesRavi = ['Harmonized_SU2C_WES_Tumor_Sample_ID_v2', 'Patient_Age_at_Diagnosis', 'Patient_Smoking_Pack_Years_Harmonized', 'Initial_Stage',
      'Agent_PD1_Atezolizumab',
      'Agent_PD1_Avelumab', 'Agent_PD1_Nivolumab',
      'Agent_PD1_Nivolumab + Ipilimumab', 'Agent_PD1_Nivolumab + Lirilumab',
      'Agent_PD1_Pembrolizumab', 'Prior_TKI', 'HLA_LOH_present', 'HLA_hom_present', 'B2M_altered', 'Ploidy', 'Purity', 'Total_amps',
      'Total_dels', 'TMB', 'TMB_clonal', 'TMB_subclonal', 'TMB_indel',
      'Neoantigens', 'Neoantigens_clonal', 'Neoantigens_subclonal']
FeaturesNeedEncodingRavi = ['Initial_Stage','Agent_PD1_Atezolizumab',
      'Agent_PD1_Avelumab', 'Agent_PD1_Nivolumab',
      'Agent_PD1_Nivolumab + Ipilimumab', 'Agent_PD1_Nivolumab + Lirilumab',
      'Agent_PD1_Pembrolizumab', 'Prior_TKI', 'HLA_LOH_present', 'HLA_hom_present', 'B2M_altered']
ProcessedClinicalDataRavi = ClinicalPreProcessing(merged_clinicalRavi, ImportantClinicalFeaturesRavi, FeaturesNeedEncodingRavi, columnsToFillNANWithMeanRavi, True)
ProcessedClinicalDataRavi = ProcessedClinicalDataRavi.rename(columns={'Harmonized_SU2C_WES_Tumor_Sample_ID_v2': 'Sample Identifier'})
ProcessedClinicalDataRavi = ProcessedClinicalDataRavi[ProcessedClinicalDataRavi['Sample Identifier'].isin(filter_idsRavi)]

In [ ]:
ProcessedClinicalDataRavi

,Sample Identifier,Patient_Age_at_Diagnosis,Patient_Smoking_Pack_Years_Harmonized,Initial_Stage,Agent_PD1_Atezolizumab,Agent_PD1_Avelumab,Agent_PD1_Nivolumab,Agent_PD1_Nivolumab + Ipilimumab,Agent_PD1_Nivolumab + Lirilumab,Agent_PD1_Pembrolizumab,...,Purity,Total_amps,Total_dels,TMB,TMB_clonal,TMB_subclonal,TMB_indel,Neoantigens,Neoantigens_clonal,Neoantigens_subclonal
0,SU2CLC-CLE-NIVO11,61,40.0,1,0,0,1,0,0,0,...,0.57,17.0,17.0,1006,217,713,3.0,154,25,116
1,SU2CLC-CLE-NIVO181,68,0.0,0,0,0,1,0,0,0,...,0.25,20.0,15.0,149,133,0,5.0,38,35,0
2,SU2CLC-CLE-NIVO191,57,15.0,1,0,0,1,0,0,0,...,0.37,10.0,10.0,602,535,15,10.0,125,110,8
3,SU2CLC-CLE-NIVO31,57,0.0,1,0,0,1,0,0,0,...,0.93,18.0,16.0,84,44,36,2.0,6,1,5
4,SU2CLC-CLE-NIVO41,67,30.0,0,0,0,1,0,0,0,...,0.62,21.0,18.0,172,106,65,0.0,24,14,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,SU2CLC-YAL-25231,63,40.0,1,1,0,0,0,0,0,...,0.59,23.0,20.0,711,455,240,18.0,79,49,29
305,SU2CLC-YAL-25251,40,40.0,1,0,0,1,0,0,0,...,0.48,22.0,19.0,576,454,105,8.0,121,92,23
306,SU2CLC-YAL-25321,75,25.0,1,0,0,0,1,0,0,...,0.25,5.0,17.0,780,703,52,8.0,169,146,18
307,SU2CLC-YAL-25371,56,12.0,1,0,0,0,0,0,1,...,0.28,8.0,11.0,291,197,69,5.0,98,59,30


In [ ]:
ProcessedGeneExpressionDataRavi = GeneExpressionPreprocessing(data_mrnaRavi, ['Name'], protein_coding_genes, 'Description', filter_genes_with_low_expression=False)
ProcessedGeneExpressionDataRavi = ProcessedGeneExpressionDataRavi[ProcessedGeneExpressionDataRavi['Sample Identifier'].isin(filter_idsRavi)]

# len(ProcessedGeneExpressionDataRavi.columns) == len(set(ProcessedGeneExpressionDataRavi.columns)) --> False
# so, I continue with removing duplicates using the helper function above

columnsRavi = ProcessedGeneExpressionDataRavi.columns.tolist()
duplicated_columnsRavi = []
for column in columnsRavi:
  if columnsRavi.count(column) > 1:
    duplicated_columnsRavi.append(column)
ProcessedGeneExpressionDataRavi = remove_duplicates(ProcessedGeneExpressionDataRavi, duplicated_columnsRavi)

# len(ProcessedGeneExpressionDataRavi.columns) == len(set(ProcessedGeneExpressionDataRavi.columns)) --> True

# ______________________________________________________________________________________

ProcessedSNPDataRavi = data_mutationsRavi[data_mutationsRavi['Hugo_Symbol'].isin(protein_coding_genes)]
ProcessedSNPDataRavi = create_count_matrix(ProcessedSNPDataRavi, 'Tumor_Sample_Barcode', 'Hugo_Symbol')
ProcessedSNPDataRavi = ProcessedSNPDataRavi[ProcessedSNPDataRavi['Sample Identifier'].isin(filter_idsRavi)]

# len(ProcessedSNPDataRavi.columns) == len(set(ProcessedSNPDataRavi.columns)) --> True

# _____________________________________________________________________________________

ProcessedCNADataRavi = data_cnaRavi[data_cnaRavi['Gene Symbol'].isin(protein_coding_genes)]

# Now I will convert the continuous CNA values to categorical...
CNAcolumns = ProcessedCNADataRavi.columns.tolist()
for column in CNAcolumns[1:]:
  ProcessedCNADataRavi[column] = ProcessedCNADataRavi[column].map(lambda x: CNA_threshold(x))

ProcessedCNADataRavi = process_and_rename_columns(ProcessedCNADataRavi, 'Gene Symbol', cat=True)
ProcessedCNADataRavi = ProcessedCNADataRavi[ProcessedCNADataRavi['Sample Identifier'].isin(filter_idsRavi)]

# len(ProcessedCNADataRavi.columns) == len(set(ProcessedCNADataRavi.columns)) --> True

In [ ]:
ProcessedGeneExpressionDataRavi

Sample,Sample Identifier,ZNF423,FRS2,C3orf62,DDRGK1,DDX6,GPT,GOLGA4,OR4A8,ZFAND1,...,MAL2,ARL14EPL,ELFN2,RABGEF1,P2RY8,MATR3,SHOX,WASH6P,SLC25A6,GTPBP6
0,SU2CLC-CLE-NIVO181,1.365510,61.86820,1.802110,42.7766,41.5127,2.64487,131.5910,0.0,8.87568,...,59.2623,0.000000,1.928850,12.45560,1.001900,264.2590,0.000000,8.54539,133.3290,16.20340
1,SU2CLC-CLE-NIVO191,4.525730,13.82350,2.610870,22.2068,68.9867,7.13729,161.6560,0.0,18.16840,...,178.2410,0.273378,0.431677,24.54440,1.149330,314.3590,0.000000,5.70080,169.6530,21.22300
2,SU2CLC-CLE-NIVO31,0.565291,21.11660,9.600540,113.7270,59.7256,28.72070,243.4570,0.0,24.47510,...,67.0554,0.090537,0.006619,31.76070,0.547486,1097.9900,0.000000,27.13040,127.7050,17.35820
3,SU2CLC-CLE-NIVO51,1.446840,6.14175,4.812200,44.2485,63.7322,17.88770,95.8694,0.0,12.13030,...,118.6770,0.000000,1.106290,10.63230,2.195860,254.3860,0.000000,18.43310,93.3517,22.60740
4,SU2CLC-CLE-NIVO91,14.245100,22.16900,3.682180,40.3596,61.8520,17.84200,154.3200,0.0,9.16326,...,36.6551,0.000000,0.262317,34.02340,1.231970,333.0740,0.000000,8.89286,190.2250,28.57390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,SU2CLC-MSK-A20131,4.741960,10.46340,3.048920,33.7511,54.0530,5.77202,103.5500,0.0,7.38677,...,50.4324,0.414689,0.708623,16.52000,2.403180,247.3110,0.007591,8.43860,143.8950,19.96380
61,SU2CLC-MSK-A20141,1.069440,23.88620,0.627531,33.3475,16.3694,23.44570,14.8649,0.0,5.52255,...,43.2260,0.199497,0.419291,3.76239,0.520609,61.4656,0.000000,7.01677,304.6930,20.21560
62,SU2CLC-MSK-A20601,0.714690,4.26514,2.035670,20.6822,29.1789,0.84012,48.8129,0.0,6.81325,...,30.6263,0.037095,0.141014,6.68851,0.881248,154.2820,0.000000,4.50207,94.8090,7.03088
63,SU2CLC-UCD-11241,2.464160,14.60760,9.365410,70.5282,52.6252,5.61953,68.5856,0.0,3.62411,...,64.6076,0.000000,2.973200,6.15680,0.308952,212.3200,0.043847,15.65110,153.3630,27.80400


In [ ]:
ProcessedSNPDataRavi

,Sample Identifier,MFN2_SNP,CROCC_SNP,RHD_SNP,TRNAU1AP_SNP,RBBP4_SNP,FLG_SNP,CLK2_SNP,GPR137B_SNP,USP39_SNP,...,FAM104A_SNP,MRPS26_SNP,FBXW12_SNP,NDUFC1_SNP,PGAM5_SNP,DNAJC15_SNP,FHL2_SNP,CHPT1_SNP,LRRC28_SNP,APOBEC3G_SNP
0,SU2CLC-CLE-NIVO181,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,SU2CLC-CLE-NIVO191,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,SU2CLC-CLE-NIVO11,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,SU2CLC-CLE-NIVO31,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,SU2CLC-CLE-NIVO41,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,SU2CLC-YAL-0101,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
305,SU2CLC-YAL-1031,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
306,SU2CLC-YAL-1041,0,0,0,0,0,2,0,0,0,...,1,1,0,0,0,0,0,0,0,0
307,SU2CLC-YAL-1081,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,1,0,0,0,0


In [ ]:
ProcessedCNADataRavi

,Sample Identifier,ACAP3_CN,ACTRT2_CN,AGRN_CN,ANKRD65_CN,ATAD3A_CN,ATAD3B_CN,ATAD3C_CN,AURKAIP1_CN,B3GALT6_CN,...,RABL2B_CN,SBF1_CN,SCO2_CN,SHANK3_CN,SYCE3_CN,TRABD_CN,TTLL8_CN,TUBGCP6_CN,TYMP_CN,ZBED4_CN
0,SU2CLC-UCD-11241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,SU2CLC-UCD-11451,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,SU2CLC-UCD-11421,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,SU2CLC-COL-10011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,SU2CLC-MSK-A20011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,SU2CLC-YAL-1041,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
305,SU2CLC-YAL-1081,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
306,SU2CLC-YAL-1111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
307,SU2CLC-MSK-A21031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# **Merging Liu and Ravi Data**

In [ ]:
'''
Merging Clinical data!!
'''

columnsCanHaveNAN = ['Agent_PD1_Atezolizumab','Agent_PD1_Avelumab', 'Agent_PD1_Nivolumab + Ipilimumab', 'Agent_PD1_Nivolumab + Lirilumab']
ProcessedClinicalDataRavi = ProcessedClinicalDataRavi.rename(columns={ 'Agent_PD1_Nivolumab':'Immunotherapy_Nivolumab',
                                                                       'Agent_PD1_Pembrolizumab':'Immunotherapy_Pembrolizumab',
                                                                       'TMB':'Total Mutations',
                                                                       'TMB_clonal':'Mutations Clonal',
                                                                       'TMB_subclonal':'Mutations Subclonal',
                                                                       'Neoantigens':'Total Neoantigen'})
clinical = pd.concat([ProcessedClinicalDataLiu, ProcessedClinicalDataRavi])
clinical1 = clinical[[col for col in clinical.columns if col not in columnsCanHaveNAN]].dropna(axis=1)
columnsCanHaveNAN_forMerge = columnsCanHaveNAN+['Sample Identifier']
clinical2 = clinical[columnsCanHaveNAN_forMerge]
for i in columnsCanHaveNAN_forMerge:
  clinical2[i].fillna(0, inplace=True)
merged_clinical = merge_on_common_column([clinical1, clinical2], 'Sample Identifier')
merged_clinical = ClinicalPreProcessing(merged_clinical, merged_clinical.columns, [], [], True)
merged_clinical.columns = merged_clinical.columns.str.replace('^(Immunotherapy_|Agent_PD1_)', '', regex=True)
merged_clinical = normalize(merged_clinical, False)

In [ ]:
merged_clinical

,Sample Identifier,Total Mutations,Mutations Clonal,Mutations Subclonal,Total Neoantigen,Purity,Ploidy,Nivolumab,Pembrolizumab,Atezolizumab,Avelumab,Nivolumab + Ipilimumab,Nivolumab + Lirilumab
0,Sample1,0.001378,0.001345,0.001913,0.001787,0.911111,0.125000,0,1,0,0,0,0
1,Sample10,0.005444,0.005379,0.004208,0.009140,0.811111,0.147917,0,1,0,0,0,0
2,Sample100,0.012266,0.010982,0.004591,0.012024,0.011111,0.216667,1,0,0,0,0,0
3,Sample104,0.007675,0.007284,0.005356,0.013162,0.844444,0.718750,0,1,0,0,0,0
4,Sample106,0.024073,0.021739,0.006886,0.029289,0.577778,0.252083,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,SU2CLC-YAL-25231,0.045786,0.050986,0.045907,0.003006,0.544444,0.437500,0,0,1,0,0,0
425,SU2CLC-YAL-25251,0.036930,0.050874,0.020084,0.004712,0.422222,0.497917,1,0,0,0,0,0
426,SU2CLC-YAL-25321,0.050312,0.078776,0.009946,0.006662,0.166667,0.195833,0,0,0,0,1,0
427,SU2CLC-YAL-25371,0.018235,0.022075,0.013198,0.003778,0.200000,0.214583,0,1,0,0,0,0


In [ ]:
'''
Merging SNP data!!
'''

# these 3 files are generated by org.Hs.eg.db R package
liu_mapping_snp = pd.read_csv('Liu/mapping_liu_snp.tsv', sep='\t')
ravi_mapping_snp = pd.read_csv('/Ravi/mapping_ravi_snp.tsv', sep='\t')
mapping_from_entrez_to_symbol_merged_snp = pd.read_csv('/Merged/mapping_from_entrez_to_symbol_for_merged_data_snp.tsv', sep='\t')

na_entrez_genes_liu_snp = liu_mapping_snp.iloc[1:, :][liu_mapping_snp.iloc[1:, :]['entrez_id'].isna()]['gene_name'].tolist()
na_entrez_genes_ravi_snp = ravi_mapping_snp.iloc[1:, :][ravi_mapping_snp.iloc[1:, :]['entrez_id'].isna()]['gene_name'].tolist()

na_entrez_genes_liu_snp = [name.replace('.', '-') for name in na_entrez_genes_liu_snp]
na_entrez_genes_ravi_snp = [name.replace('.', '-') for name in na_entrez_genes_ravi_snp]
common_genes_snp = [element for element in na_entrez_genes_ravi_snp if element in na_entrez_genes_liu_snp]

liu_mapping_snp = liu_mapping_snp.dropna(subset=['entrez_id'])
ravi_mapping_snp = ravi_mapping_snp.dropna(subset=['entrez_id'])

liu_mapping_snp['entrez_id'] = liu_mapping_snp['entrez_id'].astype('int64')
ravi_mapping_snp['entrez_id'] = ravi_mapping_snp['entrez_id'].astype('int64')

dict_liu_snp = dict(zip(liu_mapping_snp['gene_name'], liu_mapping_snp['entrez_id']))
dict_ravi_snp = dict(zip(ravi_mapping_snp['gene_name'], ravi_mapping_snp['entrez_id']))

mapped_snp_liu = ProcessedSNPDataLiu.drop(columns=[gene for gene in na_entrez_genes_liu_snp if gene in ProcessedSNPDataLiu.columns])

new_columns_snp_liu = [mapped_snp_liu.columns[0]] + [dict_liu_snp.get(gene, gene) for gene in mapped_snp_liu.columns[1:]]
mapped_snp_liu.columns = new_columns_snp_liu

mapped_snp_ravi = ProcessedSNPDataRavi.drop(columns=[gene for gene in na_entrez_genes_ravi_snp if gene in ProcessedSNPDataRavi.columns])

new_columns_snp_ravi = [mapped_snp_ravi.columns[0]] + [dict_ravi_snp.get(gene, gene) for gene in mapped_snp_ravi.columns[1:]]
mapped_snp_ravi.columns = new_columns_snp_ravi

merged_snp = pd.concat([mapped_snp_liu, mapped_snp_ravi]).dropna(axis=1)

mapping_from_entrez_to_symbol_merged_snp = mapping_from_entrez_to_symbol_merged_snp.dropna(subset=['gene_name'])
mapping_from_entrez_to_symbol_merged_snp['entrez_id'] = mapping_from_entrez_to_symbol_merged_snp['entrez_id'].astype('int64')
mapping_from_entrez_to_symbol_merged_snp['gene_name'] = mapping_from_entrez_to_symbol_merged_snp['gene_name'] + '_SNP'


merged_dict_snp = dict(zip(mapping_from_entrez_to_symbol_merged_snp['entrez_id'], mapping_from_entrez_to_symbol_merged_snp['gene_name']))

new_columns = [merged_snp.columns[0]] + [merged_dict_snp.get(gene, gene) for gene in merged_snp.columns[1:]]
merged_snp.columns = new_columns

na_ids_liu_df_snp = ProcessedSNPDataLiu[["Sample Identifier"] + common_genes_snp]
na_ids_ravi_df_snp = ProcessedSNPDataRavi[["Sample Identifier"] + common_genes_snp]
na_ids_merged_snp = pd.concat([na_ids_liu_df_snp, na_ids_ravi_df_snp]).dropna(axis=1)

In [ ]:
merged_snp_pre_normalization = merge_on_common_column([merged_snp, na_ids_merged_snp], "Sample Identifier")
merged_snp = normalize(merged_snp, True)

In [ ]:
merged_snp

,Sample Identifier,LRP1B_SNP,EIF3H_SNP,CLEC18B_SNP,BCLAF1_SNP,MUC16_SNP,RAB11FIP1_SNP,ZNF276_SNP,PMFBP1_SNP,PPFIBP2_SNP,...,KRTAP20-1_SNP,KRTAP5-1_SNP,KRTAP1-1_SNP,KRTAP10-5_SNP,HLA-DRB1_SNP,KRTAP5-8_SNP,KRTAP3-3_SNP,NKX3-1_SNP,KRTAP9-9_SNP,FAM104A_SNP
0,Sample203,0.0625,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Sample159,0.0000,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Sample108,0.0000,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Sample104,0.0000,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Sample21,0.0000,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,SU2CLC-YAL-0101,0.0000,0.0,0.0,0.00,0.012987,0.0,0.0,0.25,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
445,SU2CLC-YAL-1031,0.0000,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
446,SU2CLC-YAL-1041,0.0000,0.0,0.0,0.00,0.025974,0.5,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
447,SU2CLC-YAL-1081,0.0000,0.0,0.0,0.25,0.038961,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
'''
Merging CNA data
'''
# these 3 files are generated by org.Hs.eg.db R package
liu_mapping_cna = pd.read_csv('/Liu/mapping_liu_cna.tsv', sep='\t')
ravi_mapping_cna = pd.read_csv('/Ravi/mapping_ravi_cna.tsv', sep='\t')
mapping_from_entrez_to_symbol_merged_cna = pd.read_csv('/Merged/mapping_from_entrez_to_symbol_for_merged_data_cna.tsv', sep='\t')

na_entrez_genes_liu_cna = liu_mapping_cna.iloc[1:, :][liu_mapping_cna.iloc[1:, :]['entrez_id'].isna()]['gene_name'].tolist()
na_entrez_genes_ravi_cna = ravi_mapping_cna.iloc[1:, :][ravi_mapping_cna.iloc[1:, :]['entrez_id'].isna()]['gene_name'].tolist()

na_entrez_genes_liu_cna = [name.replace('.', '-') for name in na_entrez_genes_liu_cna]
na_entrez_genes_ravi_cna = [name.replace('.', '-') for name in na_entrez_genes_ravi_cna]
common_genes_cna = [element for element in na_entrez_genes_liu_cna if element in na_entrez_genes_ravi_cna]

liu_mapping_cna = liu_mapping_cna.dropna(subset=['entrez_id'])
ravi_mapping_cna = ravi_mapping_cna.dropna(subset=['entrez_id'])

liu_mapping_cna['entrez_id'] = liu_mapping_cna['entrez_id'].astype('int64')
ravi_mapping_cna['entrez_id'] = ravi_mapping_cna['entrez_id'].astype('int64')

liu_cbioportal_cna = dict(zip(liu_mapping_cna['gene_name'], liu_mapping_cna['entrez_id']))
dict_ravi_cna = dict(zip(ravi_mapping_cna['gene_name'], ravi_mapping_cna['entrez_id']))

mapped_cna_liu = ProcessedCNADataLiu.drop(columns=[gene for gene in na_entrez_genes_liu_cna if gene in ProcessedCNADataLiu.columns])

new_columns_cna_liu = [mapped_cna_liu.columns[0]] + [liu_cbioportal_cna.get(gene, gene) for gene in mapped_cna_liu.columns[1:]]
mapped_cna_liu.columns = new_columns_cna_liu

mapped_cna_ravi = ProcessedCNADataRavi.drop(columns=[gene for gene in na_entrez_genes_ravi_cna if gene in ProcessedCNADataRavi.columns])

new_columns_cna_ravi = [mapped_cna_ravi.columns[0]] + [dict_ravi_cna.get(gene, gene) for gene in mapped_cna_ravi.columns[1:]]
mapped_cna_ravi.columns = new_columns_cna_ravi

merged_cna = pd.concat([mapped_cna_liu, mapped_cna_ravi]).dropna(axis=1)

mapping_from_entrez_to_symbol_merged_cna = mapping_from_entrez_to_symbol_merged_cna.dropna(subset=['gene_name'])
mapping_from_entrez_to_symbol_merged_cna['entrez_id'] = mapping_from_entrez_to_symbol_merged_cna['entrez_id'].astype('int64')
mapping_from_entrez_to_symbol_merged_cna['gene_name'] = mapping_from_entrez_to_symbol_merged_cna['gene_name'] + '_CN'


merged_dict_cna = dict(zip(mapping_from_entrez_to_symbol_merged_cna['entrez_id'], mapping_from_entrez_to_symbol_merged_cna['gene_name']))

new_columns_cna = [merged_cna.columns[0]] + [merged_dict_cna.get(gene, gene) for gene in merged_cna.columns[1:]]
merged_cna.columns = new_columns_cna

na_ids_liu_df_cna = ProcessedCNADataLiu[["Sample Identifier"] + common_genes_cna]
na_ids_ravi_df_cna = ProcessedCNADataRavi[["Sample Identifier"] + common_genes_cna]
na_ids_merged_cna = pd.concat([na_ids_liu_df_cna, na_ids_ravi_df_cna]).dropna(axis=1)
merged_cna = merge_on_common_column([merged_cna, na_ids_merged_cna], "Sample Identifier")

for column in merged_cna.columns.tolist()[1:]:
        merged_cna[column] = merged_cna[column].astype('category')

In [ ]:
merged_cna

,Sample Identifier,A4GNT_CN,AADAC_CN,AADACL2_CN,AADAT_CN,AAGAB_CN,AAR2_CN,AARD_CN,AARS2_CN,AASDH_CN,...,ZWILCH_CN,ZXDC_CN,ZYX_CN,C11orf80_CN,C4orf48_CN,C9orf131_CN,FAM90A24P_CN,SOGA1_CN,SOGA3_CN,TDGF1_CN
0,Sample1,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,-1.0,...,-1.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0
1,Sample10,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,-1.0
2,Sample100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Sample104,0.0,0.0,0.0,-1.0,0.0,0.0,2.0,2.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0
4,Sample106,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,SU2CLC-YAL-1041,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0
445,SU2CLC-YAL-1081,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,-1.0
446,SU2CLC-YAL-1111,0.0,2.0,2.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,...,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.0
447,SU2CLC-MSK-A21031,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,-1.0,0.0


In [ ]:
'''
Merging Expression data!! THEN SAVING THE MERGED FILE TO BE BATCH CORRECTED. THE BATCH CORRECTED VARIABLE IS SIMPLY CALLED merged_expression
'''

liu_mapping = pd.read_csv('/Liu/mapping_liu_exp.tsv', sep='\t')
ravi_mapping = pd.read_csv('/Ravi/mapping_ravi_exp.tsv', sep='\t')
mapping_from_entrez_to_symbol_merged = pd.read_csv('/Merged/mapping_from_entrez_to_symbol_for_merged_data_exp.tsv', sep='\t')

na_entrez_genes_liu = liu_mapping.iloc[1:, :][liu_mapping.iloc[1:, :]['entrez_id'].isna()]['gene_name'].tolist()
na_entrez_genes_ravi = ravi_mapping.iloc[1:, :][ravi_mapping.iloc[1:, :]['entrez_id'].isna()]['gene_name'].tolist()

na_entrez_genes_liu = [name.replace('.', '-') for name in na_entrez_genes_liu]
na_entrez_genes_ravi = [name.replace('.', '-') for name in na_entrez_genes_ravi]
common_genes = [element for element in na_entrez_genes_ravi if element in na_entrez_genes_liu]

liu_mapping = liu_mapping.dropna(subset=['entrez_id'])
ravi_mapping = ravi_mapping.dropna(subset=['entrez_id'])

liu_mapping['entrez_id'] = liu_mapping['entrez_id'].astype('int64')
ravi_mapping['entrez_id'] = ravi_mapping['entrez_id'].astype('int64')

dict_liu = dict(zip(liu_mapping['gene_name'], liu_mapping['entrez_id']))
dict_ravi = dict(zip(ravi_mapping['gene_name'], ravi_mapping['entrez_id']))

mapped_exp_liu = ProcessedGeneExpressionDataLiu.drop(columns=[gene for gene in na_entrez_genes_liu if gene in ProcessedGeneExpressionDataLiu.columns])

new_columns = [mapped_exp_liu.columns[0]] + [dict_liu.get(gene, gene) for gene in mapped_exp_liu.columns[1:]]
mapped_exp_liu.columns = new_columns

mapped_exp_ravi = ProcessedGeneExpressionDataRavi.drop(columns=[gene for gene in na_entrez_genes_ravi if gene in ProcessedGeneExpressionDataRavi.columns])

new_columns_ravi = [mapped_exp_ravi.columns[0]] + [dict_ravi.get(gene, gene) for gene in mapped_exp_ravi.columns[1:]]
mapped_exp_ravi.columns = new_columns_ravi

merged_expression = pd.concat([mapped_exp_liu, mapped_exp_ravi]).dropna(axis=1)
mapping_from_entrez_to_symbol_merged = mapping_from_entrez_to_symbol_merged.dropna(subset=['gene_name'])
mapping_from_entrez_to_symbol_merged['entrez_id'] = mapping_from_entrez_to_symbol_merged['entrez_id'].astype('int64')
merged_dict = dict(zip(mapping_from_entrez_to_symbol_merged['entrez_id'], mapping_from_entrez_to_symbol_merged['gene_name']))

new_columns = [merged_expression.columns[0]] + [merged_dict.get(gene, gene) for gene in merged_expression.columns[1:]]
merged_expression.columns = new_columns

na_ids_liu_df = ProcessedGeneExpressionDataLiu[["Sample Identifier"] + common_genes]
na_ids_ravi_df = ProcessedGeneExpressionDataRavi[["Sample Identifier"] + common_genes]
na_ids_merged = pd.concat([na_ids_liu_df, na_ids_ravi_df]).dropna(axis=1)
merged_expression = merge_on_common_column([merged_expression, na_ids_merged], "Sample Identifier")
merged_expression = normalize(merged_expression, True)

In [ ]:
merged_expression

,Sample Identifier,ZNF423,FRS2,C3orf62,DDRGK1,DDX6,GPT,GOLGA4,ZFAND1,SERPINB12,...,HLA-DPA1,HLA-E,HLA-DQB2,HLA-C,HLA-F,MT-ND4,ERVV-2,HLA-A,ERVV-1,HLA-DRA
0,Sample100,0.264915,0.145171,0.313099,0.244932,0.401821,0.082869,0.257772,0.075181,0.0,...,0.607350,0.433734,0.007998,0.443385,0.397428,0.052920,0.000000,0.436328,0.001143,0.429330
1,Sample106,0.083568,0.137938,0.072406,0.227811,0.446215,0.053964,0.224595,0.341471,0.0,...,0.298711,0.277355,0.013462,0.314253,0.205594,0.180497,0.000000,0.256113,0.000000,0.119693
2,Sample107,0.000000,0.176433,0.000000,0.228710,0.378795,0.104098,0.741545,0.329052,0.0,...,0.000000,0.047357,0.000000,0.068351,0.042124,0.000000,0.000000,0.033294,0.000000,0.000704
3,Sample108,0.000000,0.498685,0.034920,0.219790,0.115005,0.000000,0.065338,0.314371,0.0,...,0.008301,0.003931,0.000000,0.027646,0.033587,0.462981,0.000000,0.012608,0.000000,0.007296
4,Sample10,0.222601,0.210730,0.067785,0.193039,0.426283,0.087561,0.220303,0.238790,0.0,...,0.073819,0.137843,0.003998,0.059148,0.050057,0.191295,0.039151,0.114722,0.158680,0.042010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,SU2CLC-MSK-A20131,0.187431,0.111274,0.028682,0.291911,0.273830,0.107473,0.352850,0.048198,0.0,...,0.098609,0.450486,0.078501,0.177752,0.057777,0.266494,0.001047,0.212898,0.009823,0.193671
180,SU2CLC-MSK-A20141,0.042271,0.254021,0.005903,0.288337,0.000000,0.436553,0.000000,0.036034,0.0,...,0.082042,0.384714,0.017010,0.278588,0.037850,0.003719,0.000000,0.204626,0.002126,0.169923
181,SU2CLC-MSK-A20601,0.028249,0.045358,0.019150,0.176202,0.093081,0.015643,0.135069,0.044456,0.0,...,0.317132,0.526945,0.155609,0.455180,0.142766,0.013877,0.000000,0.381890,0.000000,1.000000
182,SU2CLC-UCD-11241,0.097399,0.155346,0.088103,0.617527,0.263455,0.104634,0.213738,0.023647,0.0,...,0.139190,0.316569,0.337998,0.441700,0.064299,0.005487,0.000000,0.205381,0.000000,0.316560


In [ ]:
merged_labels = pd.concat([labelsLiu, labelsRavi])
merged_labels

,Sample Identifier,ICB Response
0,Sample1,0.0
1,Sample10,1.0
2,Sample100,1.0
4,Sample104,1.0
6,Sample106,1.0
...,...,...
388,SU2CLC-YAL-25231,0.0
389,SU2CLC-YAL-25251,0.0
390,SU2CLC-YAL-25321,1.0
391,SU2CLC-YAL-25371,0.0


# **Pre-processing Validation Data**

In [ ]:
merged_clinical_validation = merge_on_common_column([clinical_sample_validation, clinical_patient_validation], '#Patient Identifier')

# Perform one-hot encoding for 'Treatment' column
merged_clinical_validation = pd.get_dummies(merged_clinical_validation, columns=['Treatment'])
# ..........................................................................

labels_validation = merged_clinical_validation[['Sample Identifier', 'Treatment Response']]
response_mapping = {
    'response': 1,
    'nonresponse': 0
}
tumor_stage_mapping = {
    1.0: 0,
    2.0: 0,
    3.0: 1,
    4.0: 1
}
merged_clinical_validation['Tumor Stage'].replace(['1', '2', '3', '4'],
                        [1, 2, 3, 4], inplace=True)

merged_clinical_validation['Tumor Stage'] = merged_clinical_validation['Tumor Stage'].map(tumor_stage_mapping)

labels_validation['Treatment Response'] = labels_validation['Treatment Response'].map(response_mapping)
labels_validation = labels_validation.dropna(subset = ['Treatment Response', 'Sample Identifier'])
labels_validation = labels_validation.rename(columns={'Treatment Response': 'ICB Response'})
labels_validation['ICB Response'] = labels_validation['ICB Response'].astype('category')
filter_ids_validation = labels_validation['Sample Identifier']

# Preparing all the parameters needed for pre-processing Validation clinical data

columnsToFillNANWithMeanValidation = ['Serum Lactate Dehydrogenase']
ImportantClinicalFeaturesValidation = ['Sample Identifier', '#Patient Identifier', 'Serum Lactate Dehydrogenase', 'Mutation Load',
       'Neo-antigen Load', 'Tumor Stage', 'TMB (nonsynonymous)',
       'Treatment_Ipilimumab', 'Treatment_Tremelimumab', 'Treatment_Ipilimumab++vemurafenib', 'Treatment_Ipilimumab+dacarbazine']
FeaturesNeedEncodingValidation = ['Tumor Stage','Treatment_Ipilimumab', 'Treatment_Tremelimumab', 'Treatment_Ipilimumab++vemurafenib', 'Treatment_Ipilimumab+dacarbazine']
ProcessedClinicalDataValidation = ClinicalPreProcessing(merged_clinical_validation, ImportantClinicalFeaturesValidation, FeaturesNeedEncodingValidation, columnsToFillNANWithMeanValidation, True)
for i in ['Treatment_Ipilimumab', 'Treatment_Tremelimumab', 'Treatment_Ipilimumab++vemurafenib', 'Treatment_Ipilimumab+dacarbazine']:
    ProcessedClinicalDataValidation[i] = (ProcessedClinicalDataValidation[i] == True).astype(int)
    ProcessedClinicalDataValidation[i] = ProcessedClinicalDataValidation[i].astype('category')
ProcessedClinicalDataValidation = ProcessedClinicalDataValidation.drop('#Patient Identifier', axis=1)
ProcessedClinicalDataValidation = ProcessedClinicalDataValidation[ProcessedClinicalDataValidation['Sample Identifier'].isin(filter_ids_validation)]
ProcessedClinicalDataValidation = normalize(ProcessedClinicalDataValidation, False)
ProcessedClinicalDataValidation.columns = ProcessedClinicalDataValidation.columns.str.replace('^(Treatment_)', '', regex=True)

In [ ]:
ProcessedClinicalDataValidation

,Sample Identifier,Serum Lactate Dehydrogenase,Mutation Load,Neo-antigen Load,Tumor Stage,TMB (nonsynonymous),Ipilimumab,Tremelimumab,Ipilimumab++vemurafenib,Ipilimumab+dacarbazine
0,CR0095,0.008714,0.085758,0.071237,1,0.084251,0,0,0,0
1,CR04885,0.077350,0.643185,0.763309,1,0.646621,0,0,0,0
2,CR06670,0.077350,0.180092,0.137112,1,0.181602,0,0,0,0
3,CR1509,0.036929,0.164472,0.093834,1,0.158083,0,0,0,0
4,CR22640,0.077350,0.101685,0.083110,1,0.102709,0,0,0,0
5,CR3665,0.062656,0.042573,0.017235,1,0.042572,0,0,0,0
6,CR4880,0.000000,0.786524,0.775948,1,0.748139,0,0,0,0
7,CR6126,0.049378,0.171516,0.172348,1,0.169098,0,0,0,0
8,CR6161,0.077350,0.284227,0.367292,1,0.286692,0,0,0,0
9,CR7623,0.109959,0.153446,0.162773,1,0.147365,0,0,0,0


In [ ]:
mapping_validation.rename(columns={"entrez_id": "Entrez_Gene_Id"}, inplace=True)
merged_df_validation = pd.merge(data_mrna_validation, mapping_validation, on="Entrez_Gene_Id", how="inner")
if "entrez_id" in merged_df_validation.columns:
  merged_df_validation = merged_df_validation.drop(columns=['entrez_id'])
merged_df_validation = merged_df_validation.dropna(subset=['gene_name'])
merged_df_validation = merged_df_validation.rename(columns={'gene_name': 'Hugo_Symbol'})

In [ ]:
ProcessedGeneExpressionDataValidation = GeneExpressionPreprocessing(merged_df_validation, ['Entrez_Gene_Id'], merged_df_validation['Hugo_Symbol'].tolist(), 'Hugo_Symbol', tpm=False, filter_genes_with_low_expression=False)
ProcessedGeneExpressionDataValidation = ProcessedGeneExpressionDataValidation[ProcessedGeneExpressionDataValidation['Sample Identifier'].isin(filter_ids_validation)]

#len(ProcessedGeneExpressionDataValidation.columns) == len(set(ProcessedGeneExpressionDataValidation.columns)) --> False
# so, I continue with removing duplicates using the helper function above

columnsValidation = ProcessedGeneExpressionDataValidation.columns.tolist()
duplicated_columnsValidation = []
for column in columnsValidation:
  if columnsValidation.count(column) > 1:
    duplicated_columnsValidation.append(column)

ProcessedGeneExpressionDataValidation = remove_duplicates(ProcessedGeneExpressionDataValidation, duplicated_columnsValidation)
ProcessedGeneExpressionDataValidation = ProcessedGeneExpressionDataValidation.T.drop_duplicates().T

for col in ProcessedGeneExpressionDataValidation.columns[1:]:
    ProcessedGeneExpressionDataValidation[col] = pd.to_numeric(ProcessedGeneExpressionDataValidation[col], errors='coerce')

for row in ProcessedGeneExpressionDataValidation.index:
      row_sum = ProcessedGeneExpressionDataValidation.iloc[row, 1:].sum()
      for column in ProcessedGeneExpressionDataValidation.columns[1:]:
        ProcessedGeneExpressionDataValidation.loc[row, column] = (ProcessedGeneExpressionDataValidation.loc[row, column] / row_sum) * 10**6

filtered_columns = ['Sample Identifier'] + [col for col in set(ProcessedGeneExpressionDataValidation.columns[1:]) if col in protein_coding_genes]
ProcessedGeneExpressionDataValidation = ProcessedGeneExpressionDataValidation[filtered_columns]

ProcessedGeneExpressionDataValidation = normalize(ProcessedGeneExpressionDataValidation, True)

# len(ProcessedGeneExpressionDataValidation.columns) == len(set(ProcessedGeneExpressionDataValidation.columns)) --> True
# # _____________________________________________________

ProcessedSNPDataValidation = data_mutations_validation[data_mutations_validation['Hugo_Symbol'].isin(protein_coding_genes)]
ProcessedSNPDataValidation = create_count_matrix(ProcessedSNPDataValidation, 'Tumor_Sample_Barcode', 'Hugo_Symbol')
ProcessedSNPDataValidation = ProcessedSNPDataValidation[ProcessedSNPDataValidation['Sample Identifier'].isin(filter_ids_validation)]
ProcessedSNPDataValidation = normalize(ProcessedSNPDataValidation, True)

# len(ProcessedSNPDataValidation.columns) == len(set(ProcessedSNPDataValidation.columns)) --> True
# # _____________________________________________________

ProcessedCNADataValidation = data_cna_validation[data_cna_validation['Hugo_Symbol'].isin(protein_coding_genes)]
ProcessedCNADataValidation = process_and_rename_columns(ProcessedCNADataValidation, 'Hugo_Symbol')
ProcessedCNADataValidation = ProcessedCNADataValidation[ProcessedCNADataValidation['Sample Identifier'].isin(filter_ids_validation)]

# len(ProcessedCNADataValidation.columns) == len(set(ProcessedCNADataValidation.columns)) --> True

In [ ]:
ProcessedGeneExpressionDataValidation

Sample,Sample Identifier,FADS3,CEP170B,CCN5,ARHGAP31,OR13C2,ZNF229,SLC5A9,VSIG10L,IRGC,...,LRCH1,ZMIZ1,NXF1,TSEN15,VCAN,DYNLT5,CST8,KCNRG,LAG3,INSYN2B
0,CR1509,0.479226,0.570287,0.258554,0.765591,0.0,0.184739,0.040371,0.170504,0.000000,...,0.471997,0.469268,0.420106,0.473593,0.062926,0.077983,0.0,0.000000,0.652211,0.160399
1,CR6126,0.000000,0.312470,0.343894,0.297922,0.0,0.012012,0.010011,1.000000,0.000000,...,0.798056,0.106150,0.000000,0.815015,0.043160,0.056939,0.0,0.288957,0.019404,0.021017
2,CR7623,0.388763,0.101265,0.138282,0.208082,0.0,0.211843,0.013762,0.050040,0.000000,...,0.889956,0.434445,0.405167,0.561753,1.000000,0.044980,0.0,0.765386,0.025203,0.227790
3,CR9699,0.251831,0.080503,0.004924,0.169669,0.0,0.058880,0.115568,0.069137,0.000000,...,0.675378,0.118393,0.477526,0.612061,0.005993,0.039767,0.0,0.208108,0.184674,0.001898
4,LSD0167,0.208482,0.218002,0.009324,0.012565,0.0,0.575296,1.000000,0.132538,0.000000,...,0.647059,0.149660,0.580702,0.488548,0.360747,1.000000,1.0,0.415333,0.118210,0.101686
5,NR3549,0.365787,0.689691,0.091297,1.000000,1.0,0.067397,0.122178,0.000000,0.000000,...,0.737764,1.000000,0.617473,0.623136,0.036707,0.030378,0.0,0.721496,0.024735,0.172131
6,NR4631,0.706141,0.143772,0.053971,0.655902,0.0,0.010406,0.037819,0.093435,0.311866,...,0.290621,0.354110,0.607122,0.918281,0.000000,0.012723,0.0,1.000000,0.001111,0.063648
7,NR4810,0.710622,0.666451,0.000000,0.892446,0.0,0.039031,0.013745,0.078286,0.000000,...,0.920525,0.010837,0.349998,0.762642,0.126218,0.040445,0.0,0.580630,0.004200,0.080756
8,NR5784,0.292825,0.000000,1.000000,0.416173,0.0,0.057143,0.025947,0.218341,0.931917,...,0.376768,0.210466,0.631672,0.000000,0.190785,0.247618,0.0,0.373107,0.035960,0.038502
9,NR9705,0.009340,0.065835,0.139187,0.329192,0.0,0.112106,0.089447,0.046379,0.828950,...,0.714105,0.386885,1.000000,1.000000,0.956816,0.265370,0.0,0.217677,0.000000,0.000000


In [ ]:
ProcessedSNPDataValidation

,Sample Identifier,MTARC2_SNP,SELENOF_SNP,A1CF_SNP,A2M_SNP,A2ML1_SNP,A3GALT2_SNP,AAAS_SNP,AADAC_SNP,AADACL2_SNP,...,NRSN1_SNP,PLCXD2_SNP,RBM15B_SNP,RUFY2_SNP,SAE1_SNP,SLC46A1_SNP,CT83_SNP,TMEM8B_SNP,UTS2_SNP,ZNF511_SNP
1,LSDNR9298,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,CR0095,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CRNR0244,0.0,0.0,0.5,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,CR22640,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,CR9699,0.0,0.0,0.5,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,NR8815,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,CR04885,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,CR4880,0.0,0.0,0.5,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
10,NR4810,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,LSDNR1120,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
ProcessedCNADataValidation

,Sample Identifier,OR4F16_CN,OR4F29_CN,OR4F3_CN,OR4F5_CN,SAMD11_CN,KLHL17_CN,NOC2L_CN,PLEKHN1_CN,HES4_CN,...,PLXNB2_CN,PPP6R2_CN,RABL2B_CN,SBF1_CN,SCO2_CN,SHANK3_CN,SYCE3_CN,TRABD_CN,TUBGCP6_CN,TYMP_CN
1,CR6161,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,CR0095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,CR04885,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,CR06670,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,CR1509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,CR22640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,CR3665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,CR4880,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,CR6126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,CR9306,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# **Feature Selection**

## **Random Forest**

In [ ]:
'''
A function that implements feature selection with random forest
It builds a RF model and retrieves the feature importances
Then we sort by importance in descending order and get the top n genes on the condition that they are larger than 0.
It returns a random forest importance dataframe with two columns: gene and importance
'''

def feature_selection_with_rf(df, label_df, commoncolumn='Sample Identifier', labelID ='ICB Response', n_feature=175):
    merged_df = pd.merge(df, label_df, on=commoncolumn)
    X = merged_df.drop(columns=[commoncolumn, labelID])
    y = merged_df[labelID]

    rf = RandomForestClassifier(random_state=42, n_jobs=-1)

    param_grid = {
        'n_estimators': [10, 100, 1000],
        'min_samples_split': [2, 5, 10],
        'criterion': ['gini', 'entropy'],
    }

    grid_search = GridSearchCV(rf, param_grid, cv=None, scoring='accuracy')
    grid_search.fit(X, y)
    best_rf = grid_search.best_estimator_

    rf_importances = best_rf.feature_importances_
    rf_importances_df = pd.DataFrame({
        'Feature': X.columns,
        'Importance': rf_importances
    })
    rf_importances_df = rf_importances_df.sort_values(by='Importance', ascending=False)
    rf_importances_df = rf_importances_df.head(n_feature)
    rf_importances_df = rf_importances_df[rf_importances_df['Importance'] > 0]
    return rf_importances_df

## **Recursive Feature Elimination**

In [ ]:
def recursive_feature_elimination(df, labels_df, commoncolumn='Sample Identifier', labelID ='ICB Response', n_features_to_select=175, step=100):
    merged_df = pd.merge(df, labels_df, on=commoncolumn)
    X = merged_df.drop([commoncolumn, labelID], axis=1)
    y = merged_df[labelID]
    logreg = LogisticRegression(max_iter = 1000, n_jobs=-1)
    rfe = RFE(estimator=logreg, n_features_to_select=n_features_to_select, step=step)

    param_grid = {
        'estimator__C': [0.01, 0.1, 1]
    }
    grid_search = GridSearchCV(rfe, param_grid, cv=None, scoring='accuracy')
    grid_search.fit(X, y)

    selected_features = pd.DataFrame({'Feature': X.columns,
                                      'Selected': grid_search.best_estimator_.support_,
                                      'Ranking': grid_search.best_estimator_.ranking_})
    s = selected_features.sort_values(by='Ranking', ascending=True)

    # Selecting genes with ranking = 1, that is the top n_features_to_select genes
    s = s[s['Ranking'] == 1]
    return s

## **ANOVA F-statistic**

In [ ]:
def anova(df, labels_df, commoncolumn='Sample Identifier', n_feature=175, labelID ='ICB Response'):
  merged_df = pd.merge(df, labels_df, on=commoncolumn)
  X = merged_df.drop([commoncolumn, labelID], axis=1)
  y = merged_df[labelID]

  f_scores, _ = f_classif(X, y)

  f_df = pd.DataFrame({'Feature': X.columns, 'F_Score': f_scores})

  f_df = f_df.sort_values(by='F_Score', ascending=False)

  f_df = f_df.head(n_feature)

  return f_df

## **Mutual Information**

In [ ]:
def entropy(df, labels_df, commoncolumn='Sample Identifier', labelID ='ICB Response', top_k=175):

  merged_df = pd.merge(df, labels_df, on=commoncolumn)
  X = merged_df.drop([commoncolumn, labelID], axis=1)
  y = merged_df[labelID]

  info_gain = mutual_info_classif(X, y, discrete_features='auto')

  info_gain_df = pd.DataFrame({'Feature': X.columns, 'Information Gain': info_gain})
  info_gain_df = info_gain_df.sort_values(by='Information Gain', ascending=False).reset_index(drop=True)

  if top_k is not None:
      info_gain_df = info_gain_df.head(top_k)
      info_gain_df = info_gain_df[info_gain_df['Information Gain'] > 0]

  return info_gain_df

## **Feature selection for the integrative data types altogether!**

In [ ]:
integrative = merge_dataframes([merged_cna, merged_snp, merged_expression, merged_clinical, merged_cibersort], "Sample Identifier")

rf_integrative = feature_selection_with_rf(integrative, merged_labels, n_feature=175)
rfe_integrative = recursive_feature_elimination(integrative, merged_labels, step=100, n_features_to_select=175)
fdr_integrative = fdr(integrative, merged_labels, n_feature=175)
MI_integrative = entropy(integrative, merged_labels, top_k=175)

In [ ]:
'''
Union features
'''

top_rf = rf_integrative['Feature'].head(125).tolist()
top_rfe = rfe_integrative['Feature'].head(125).tolist()
top_fdr = fdr_integrative['Feature'].head(125).tolist()
top_MI = MI_integrative['Feature'].head(125).tolist()

top_features_integrative = set(top_rf + top_fdr + top_MI + top_rfe)

exp_f = []
snp_f = []
cna_f = []
clinical_f = []
immune_f = []
for f in top_features_integrative:
  if f in merged_expression.columns:
    exp_f.append(f)
  elif f in merged_snp.columns:
    snp_f.append(f)
  elif f in merged_cna.columns:
    cna_f.append(f)
  elif f in merged_clinical.columns:
    clinical_f.append(f)
  else:
    immune_f.append(f)

In [ ]:
if immune_f:
  immune_subset = merged_cibersort[['Sample Identifier'] + immune_f]
else:
  immune_subset = []

if clinical_f:
  clinical_subset = merged_clinical[['Sample Identifier'] + clinical_f]
else:
  clinical_subset = []

exp_subset = merged_expression[['Sample Identifier'] + exp_f]
snp_subset = merged_snp[['Sample Identifier'] + snp_f]
cna_subset = merged_cna[['Sample Identifier'] + cna_f]

# **Data pre-processing for train/test compatability upon validation**

In [ ]:
'''
Preparing validation from the integrative FS model (merged)
'''

ClinicalDataValidation = ClinicalDataValidation.rename(columns={'Mutation Load': 'Total Mutations', 'Neo-antigen Load': 'Total Neoantigen'})

cols_cna_common = []
for col in cna_f:
  if col in CNADataValidation.columns:
    cols_cna_common.append(col)

cols_snp_common = []
for col in snp_f:
  if col in SNPDataValidation.columns:
    cols_snp_common.append(col)

cols_exp_common = []
for col in exp_f:
  if col in GeneExpressionDataValidation.columns:
    cols_exp_common.append(col)

cols_clinical_common = []
for col in clinical_f:
  if col in ClinicalDataValidation.columns:
    cols_clinical_common.append(col)


filtered_CNA_Data_Validation = CNADataValidation[['Sample Identifier'] + cols_cna_common]
filtered_SNP_Data_Validation = SNPDataValidation[['Sample Identifier'] + cols_snp_common]
filtered_Expression_Data_Validation = GeneExpressionDataValidation[['Sample Identifier'] + cols_exp_common]
if immune_f:
  filtered_Immune_Validation = cibersort_validation[['Sample Identifier'] + immune_f]
filtered_clinical_Validation = ClinicalDataValidation[['Sample Identifier'] + cols_clinical_common]

filtered_CNA_Data_Training = merged_cna[['Sample Identifier'] + cols_cna_common]
filtered_SNP_Data_Training = merged_snp[['Sample Identifier'] + cols_snp_common]
filtered_Expression_Data_Training = merged_expression[['Sample Identifier'] + cols_exp_common]
if immune_f:
  filtered_Immune_Training = merged_cibersort[['Sample Identifier'] + immune_f]
filtered_clinical_Training = merged_clinical[['Sample Identifier'] + cols_clinical_common]

for column in filtered_CNA_Data_Validation.columns.tolist()[1:]:
        filtered_CNA_Data_Validation[column] = filtered_CNA_Data_Validation[column].astype('category')

for column in filtered_CNA_Data_Training.columns.tolist()[1:]:
        filtered_CNA_Data_Training[column] = filtered_CNA_Data_Training[column].astype('category')

# **Classifiers**

## **Random Forest**

In [ ]:
'''
A function for building an optimized random forest model. All it needs is a dataframe for the features and another one for the labels
It returns a dataframe that contains all the results (accuracies)
'''

def randomforest_classifier(X, y, training_for_validation=None, testing_for_validation=None):
  results_dataset = None

  param_grid = {
      'n_estimators': [10, 100, 1000],
      'criterion': ['gini', 'entropy']
  }

  clf = RandomForestClassifier(random_state=42)
  grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

  if training_for_validation is None and testing_for_validation is None:
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
      grid_search.fit(X_train, y_train)

      best_clf = grid_search.best_estimator_

      y_pred = best_clf.predict(X_test)
      test_accuracy = accuracy_score(y_test, y_pred)
      f1 = f1_score(y_test, y_pred)
      y_pred_proba = best_clf.predict_proba(X_test)[:, 1]
      roc_auc = roc_auc_score(y_test, y_pred_proba)

      results_dataset = {
          'Test Accuracy': test_accuracy,
          'F1 Score': f1,
          'AUC Accuracy': roc_auc
      }
      print("Best Parameters:", grid_search.best_params_)
      return results_dataset, y_test, y_pred

  else:
      training_X = training_for_validation.iloc[:, 1:-1]
      training_y = training_for_validation.iloc[:, -1]
      testing_X = testing_for_validation.iloc[:, 1:-1]
      testing_y = testing_for_validation.iloc[:, -1]

      grid_search.fit(training_X, training_y)
      best_clf = grid_search.best_estimator_

      y_pred_validation = best_clf.predict(testing_X)
      test_accuracy_validation = accuracy_score(testing_y, y_pred_validation)
      f1_validation = f1_score(testing_y, y_pred_validation)
      y_pred_proba_validation = best_clf.predict_proba(testing_X)[:, 1]
      roc_auc_validation = roc_auc_score(testing_y, y_pred_proba_validation)

      results_dataset = {
          'Validation Test Accuracy': test_accuracy_validation,
          'Validation F1 Score': f1_validation,
          'Validation AUC Accuracy': roc_auc_validation
      }
      print("Best Parameters:", grid_search.best_params_)
      return results_dataset, testing_y, y_pred_validation

## **Support Vector Machine**

In [ ]:
def svm_classifier(X=None, y=None, training_for_validation=None, testing_for_validation=None):
  param_grid = {
      'C': [0.1, 1, 10, 100],
      'kernel': ['linear', 'rbf', 'poly']
  }

  clf = SVC(probability=True)  # probability=True is needed for predict_proba
  grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

  results_dataset = None

  if training_for_validation is None and testing_for_validation is None:
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
      grid_search.fit(X_train, y_train)

      best_clf = grid_search.best_estimator_

      y_pred = best_clf.predict(X_test)
      test_accuracy = accuracy_score(y_test, y_pred)
      f1 = f1_score(y_test, y_pred)
      y_pred_proba = best_clf.predict_proba(X_test)[:, 1]
      roc_auc = roc_auc_score(y_test, y_pred_proba)

      results_dataset = {
          'Test Accuracy': test_accuracy,
          'F1 Score': f1,
          'AUC Accuracy': roc_auc
      }
      print("Best Parameters:", grid_search.best_params_)
      return results_dataset, y_test, y_pred

  else:
      training_X = training_for_validation.iloc[:, 1:-1]
      training_y = training_for_validation.iloc[:, -1]
      testing_X = testing_for_validation.iloc[:, 1:-1]
      testing_y = testing_for_validation.iloc[:, -1]

      grid_search.fit(training_X, training_y)
      best_clf = grid_search.best_estimator_

      y_pred_validation = best_clf.predict(testing_X)
      test_accuracy_validation = accuracy_score(testing_y, y_pred_validation)
      f1_validation = f1_score(testing_y, y_pred_validation)
      y_pred_proba_validation = best_clf.predict_proba(testing_X)[:, 1]
      roc_auc_validation = roc_auc_score(testing_y, y_pred_proba_validation)

      results_dataset = {
          'Validation Test Accuracy': test_accuracy_validation,
          'Validation F1 Score': f1_validation,
          'Validation AUC Accuracy': roc_auc_validation
      }
      print("Best Parameters:", grid_search.best_params_)
      return results_dataset, testing_y, y_pred_validation

## **Logistic Regression**

In [ ]:
def logistic_regression_classifier(X, y, training_for_validation, testing_for_validation):
    clf = LogisticRegression(max_iter=1000, random_state=42)
    param_grid = {
      'C': [0.001, 0.01, 0.1, 1, 10, 100],
      'solver': ['newton-cg', 'liblinear'],
      'class_weight': [None, 'balanced'],
    }

    results_dataset = None

    if training_for_validation is None and testing_for_validation is None:
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

      grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

      grid_search.fit(X_train, y_train)
      best_clf = grid_search.best_estimator_

      y_pred = best_clf.predict(X_test)
      test_accuracy = accuracy_score(y_test, y_pred)
      f1 = f1_score(y_test, y_pred)
      y_pred_proba = best_clf.predict_proba(X_test)[:, 1]
      roc_auc = roc_auc_score(y_test, y_pred_proba)

      results_dataset = {
      'Test Accuracy': test_accuracy,
      'F1 Score': f1,
      'AUC Accuracy': roc_auc
    }
      print("Best Parameters:", grid_search.best_params_)
      return results_dataset, y_test, y_pred

    else:
      training_for_validation_x = training_for_validation.iloc[:, 1:-1]
      training_for_validation_y = training_for_validation.iloc[:, -1]

      testing_for_validation_x = testing_for_validation.iloc[:, 1:-1]
      testing_for_validation_y = testing_for_validation.iloc[:, -1]


      grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
      grid_search.fit(training_for_validation_x, training_for_validation_y)
      best_clf = grid_search.best_estimator_

      y_pred_validation = best_clf.predict(testing_for_validation_x)
      test_accuracy_validation = accuracy_score(testing_for_validation_y, y_pred_validation)
      f1_validation = f1_score(testing_for_validation_y, y_pred_validation)
      y_pred_proba_validation = best_clf.predict_proba(testing_for_validation_x)[:, 1]
      roc_auc_validation = roc_auc_score(testing_for_validation_y, y_pred_proba_validation)

      results_dataset = {
          'Validation Test Accuracy': test_accuracy_validation,
          'Validation F1 Score': f1_validation,
          'Validation AUC Accuracy': roc_auc_validation
      }
      print("Best Parameters:", grid_search.best_params_)
      return results_dataset, testing_for_validation_y, y_pred_validation

## **For quickly building a simple model from our filtered dataframes.**

In [ ]:
data = [clinical_subset, immune_subset, snp_subset, exp_subset, cna_subset, merged_labels]
model = merge_dataframes(data, 'Sample Identifier')
result = logistic_regression_classifier(model.iloc[:, 1:-1], model.iloc[:, -1], None, None)
result_df = pd.DataFrame([result[0]])
result_df

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best Parameters: {'C': 10, 'class_weight': None, 'solver': 'liblinear'}


,Mean CV Accuracy,Test Accuracy,F1 Score,AUC Accuracy
0,0.820197,0.864865,0.8,0.909091


## **External Validation Dataset**

In [ ]:
data_training = [filtered_clinical_Training, filtered_Immune_Training, filtered_SNP_Data_Training, filtered_Expression_Data_Training, filtered_CNA_Data_Training, merged_labels]
data_testing = [filtered_clinical_Validation, filtered_Immune_Validation, filtered_SNP_Data_Validation, filtered_Expression_Data_Validation, filtered_CNA_Data_Validation, LabelsValidation]
model_training = merge_dataframes(data_training, 'Sample Identifier')
model_testing = merge_dataframes(data_testing, 'Sample Identifier')
result = logistic_regression_classifier(None, None, model_training, model_testing)
result_df = pd.DataFrame([result[0]])
result_df

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best Parameters: {'C': 1, 'class_weight': None, 'solver': 'liblinear'}


,Validation Mean CV Accuracy,Validation Test Accuracy,Validation F1 Score,Validation AUC Accuracy
0,0.883978,0.75,0.6,0.666667


# **Visualization of SNP and CNA response-associated genes among responders and non-responders**

In [ ]:
merged_snp = merge_dataframes([merged_snp_pre_normalization, merged_labels], 'Sample Identifier')
merged_cna = merge_dataframes([merged_cna, merged_labels], 'Sample Identifier')

In [ ]:
def preprocess_snp(df):
    features = df.iloc[:, 1:-1]
    label = df.iloc[:, -1]

    count_row = (features > 0).sum(axis=0)

    sum_row = features.sum(axis=0)

    processed_df = pd.DataFrame([count_row, sum_row], index=['count', 'sum'])

    processed_df = processed_df.loc[:, processed_df.loc['count'] > 0]

    processed_df = processed_df.T.reset_index()

    processed_df.columns = ['Symbol', 'count', 'sum']

    processed_df['Symbol'] = processed_df['Symbol'].str.replace('_SNP', '')
    return processed_df

def preprocess_cna(df):
    features = df.iloc[:, 1:-1].apply(pd.to_numeric)
    label = df.iloc[:, -1]

    count_nonzero_row = (features != 0).sum(axis=0)

    sum_pos_row = features[features > 0].sum(axis=0)

    sum_neg_row = features[features < 0].sum(axis=0)

    processed_df = pd.DataFrame([count_nonzero_row, sum_pos_row, sum_neg_row], index=['count', 'sum_pos', 'sum_neg'])

    processed_df = processed_df.loc[:, processed_df.loc['count'] > 0]

    processed_df = processed_df.T.reset_index()

    processed_df.columns = ['Symbol', 'count', 'sum_pos', 'sum_neg']

    processed_df['Symbol'] = processed_df['Symbol'].str.replace('_CN', '')
    return processed_df

def process_all_dataframes(df1, df2):
    df1_class0 = df1[df1.iloc[:, -1] == 0]
    df1_class1 = df1[df1.iloc[:, -1] == 1]
    df2_class0 = df2[df2.iloc[:, -1] == 0]
    df2_class1 = df2[df2.iloc[:, -1] == 1]

    df1_class0_processed = preprocess_snp(df1_class0)
    df1_class1_processed = preprocess_snp(df1_class1)
    df2_class0_processed = preprocess_cna(df2_class0)
    df2_class1_processed = preprocess_cna(df2_class1)

    return df1_class0_processed, df1_class1_processed, df2_class0_processed, df2_class1_processed

In [ ]:
def visualize_snp(df_class0, df_class1):
    df_class0 = df_class0.set_index('Symbol')
    df_class1 = df_class1.set_index('Symbol')

    plt.figure(figsize=(14, 8))
    combined_counts = pd.DataFrame({
        'Non-responders': df_class0['count'],
        'Responders': df_class1['count']
    })
    combined_counts.plot(kind='bar', ax=plt.gca(), color=['red', 'blue'])

    plt.title('SNP Occurences')
    plt.xticks(rotation=90)
    plt.xlabel('Gene Symbol')
    plt.ylabel('Occurences')
    plt.legend(loc='upper right')
    plt.show()

    plt.figure(figsize=(14, 8))
    combined_sums = pd.DataFrame({
        'Non-responders': df_class0['sum'],
        'Responders': df_class1['sum']
    })
    combined_sums.plot(kind='bar', ax=plt.gca(), color=['red', 'blue'])

    plt.title('Total Number of SNP Mutations')
    plt.xticks(rotation=90)
    plt.xlabel('Gene Symbol')
    plt.ylabel('Sum')
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
def visualize_cna(df_class0, df_class1):
    df_class0 = df_class0.set_index('Symbol')
    df_class1 = df_class1.set_index('Symbol')

    plt.figure(figsize=(14, 8))
    combined_counts = pd.DataFrame({
        'Non-responders': df_class0['count'],
        'Responders': df_class1['count']
    })
    combined_counts.plot(kind='bar', ax=plt.gca(), color=['red', 'blue'])

    plt.title('CNA Occurences')
    plt.xticks(rotation=90)
    plt.xlabel('Gene Symbol')
    plt.ylabel('Count')
    plt.legend(loc='upper right')
    plt.show()

    plt.figure(figsize=(14, 8))
    combined_sums = pd.DataFrame({
        'Non-responders': df_class0['sum_pos'],
        'Responders': df_class1['sum_pos']
    })
    combined_sums.plot(kind='bar', ax=plt.gca(), color=['red', 'blue'])

    plt.title('Total Number of Amplifications')
    plt.xticks(rotation=90)
    plt.xlabel('Gene Symbol')
    plt.ylabel('Sum of amplifications')
    plt.legend(loc='upper right')
    plt.show()

    plt.figure(figsize=(14, 8))
    combined_sums = pd.DataFrame({
        'Non-responders': df_class0['sum_neg'],
        'Responders': df_class1['sum_neg']
    })
    combined_sums.plot(kind='bar', ax=plt.gca(), color=['red', 'blue'])

    plt.title('Total number of Deletions')
    plt.xticks(rotation=90)
    plt.xlabel('Gene Symbol')
    plt.ylabel('Sum of deletions')
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
def preprocess_gene_relations(df):
    gene_pairs_count = defaultdict(int)

    for index, row in df.iterrows():
        mutated_genes = row.index[row > 0].tolist()
        for gene1, gene2 in combinations(mutated_genes, 2):
            pair = tuple(sorted([gene1, gene2]))
            gene_pairs_count[pair] += 1

    relations = pd.DataFrame([(gene1, gene2, count) for (gene1, gene2), count in gene_pairs_count.items()],
                             columns=['Gene1', 'Gene2', 'count']).sort_values(by='count', ascending=False)
    relations['Gene1'] = relations['Gene1'].str.replace('_SNP', '')
    relations['Gene2'] = relations['Gene2'].str.replace('_SNP', '')
    return relations

In [ ]:
filtered_merged_snp = merged_snp[['Sample Identifier'] + snp_f + ['ICB Response']]
filtered_merged_cna = merged_cna[['Sample Identifier'] + cna_f + ['ICB Response']]

snp_responders, snp_nonresponders, cna_responders, cna_nonresponders = process_all_dataframes(filtered_merged_snp, filtered_merged_cna)

In [ ]:
plt.figure(figsize=(14, 6))

# Plot histograms for the 'sum_pos' column
plt.subplot(1, 3, 1)
sns.histplot(cna_responders['sum_pos'], color='blue', bins=20, alpha=0.5, label='Responders', stat='density', kde=True)
sns.histplot(cna_nonresponders['sum_pos'], color='red', bins=20, alpha=0.5, label='Non-responders', stat='density', kde=True)
plt.xlabel('Sum of Amplifications')
plt.ylabel('Frequency')
plt.title('Distribution of Total Number of Amplifications')
plt.legend(loc='upper right')

# Plot histograms for the 'sum_neg' column
plt.subplot(1, 3, 2)
sns.histplot(cna_responders['sum_neg'], color='blue', bins=20, alpha=0.5, label='Responders', stat='density', kde=True)
sns.histplot(cna_nonresponders['sum_neg'], color='red', bins=20, alpha=0.5, label='Non-responders', stat='density', kde=True)
plt.xlabel('Sum of Deletions')
plt.ylabel('Frequency')
plt.title('Distribution of Total Number of Deletions')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
filtered_nonresponders = filtered_merged_snp[filtered_merged_snp['ICB Response'] == 0]
filtered_responders = filtered_merged_snp[filtered_merged_snp['ICB Response'] == 1]
relations_nonresponders = preprocess_gene_relations(filtered_nonresponders.iloc[:, 1:-1])
relations_responders = preprocess_gene_relations(filtered_responders.iloc[:, 1:-1])